<a href="https://colab.research.google.com/github/md-labs/covid19-kaggle/blob/hong/micro_scorers/micro_scorers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install transformers
!pip install rpforest

     |████████████████████████████████| 573kB 2.7MB/s 
     |████████████████████████████████| 3.7MB 52.9MB/s 
     |████████████████████████████████| 1.0MB 22.9MB/s 
     |████████████████████████████████| 890kB 45.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.41-cp36-none-any.whl size=893334 sha256=413bafb07ef473f111b79725515b1c3a367ebb457d8396e6be925816f9c2561c
  Stored in directory: /root/.cache/pip/wheels/22/5a/d4/b020a81249de7dc63758a34222feaa668dbe8ebfe9170cc9b1
Successfully built sacremoses
     |████████████████████████████████| 4.4MB 4.8MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for rpforest: filename=rpforest-1.6-cp36-cp36m-linux_x86_64.whl size=687007 sha256=662a8c826ff6fa34a1abeacf2cbc63e092890f6bb2c9ae61c5e195e0f66b3c25
  Stored in directory: /root/.cache/pip/wheels/c1/e0/18/4b85e59d998a836b344d1225e28d80e9289

In [0]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import euclidean, cosine
from transformers import RobertaTokenizer, RobertaModel
import torch
from torch.utils.data import TensorDataset, DataLoader
from scipy.special import softmax
import numpy as np

from tqdm import tqdm
import math

##1. Load dataset

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
root_dir = '/content/drive/Shared drives/COVID-19-Research/data/'
df = pd.read_csv(f'{root_dir}/metadata.csv', header=0, usecols=['title', 'abstract', 'journal'])

In [0]:
df['title'].isnull().sum()

158

In [0]:
print(len(df))
df = df[df['title'].notna()]
df = df[df['abstract'].notna()]
print(len(df))

51078
42203


In [0]:
df['title'].fillna('', inplace=True)
# combine title and abstract
df['title+abstract'] = df['title'] + '\n' + df['abstract']

In [0]:
print(len(df))
print(len(df[df['title+abstract']=='']))

df.head()

42203
0


,title,abstract,journal,title+abstract
0,Airborne rhinovirus detection and effect of ul...,"BACKGROUND: Rhinovirus, the most common cause ...",BMC Public Health,Airborne rhinovirus detection and effect of ul...
1,Discovering human history from stomach bacteria,Recent analyses of human pathogens have reveal...,Genome Biol,Discovering human history from stomach bacteri...
2,A new recruit for the army of the men of death,"The army of the men of death, in John Bunyan's...",Genome Biol,A new recruit for the army of the men of death...
3,Association of HLA class I with severe acute r...,BACKGROUND: The human leukocyte antigen (HLA) ...,BMC Med Genet,Association of HLA class I with severe acute r...
4,A double epidemic model for the SARS propagation,BACKGROUND: An epidemic of a Severe Acute Resp...,BMC Infect Dis,A double epidemic model for the SARS propagati...


In [0]:
df.reset_index(drop=True, inplace=True)
df.head()

,title,abstract,journal,title+abstract
0,Airborne rhinovirus detection and effect of ul...,"BACKGROUND: Rhinovirus, the most common cause ...",BMC Public Health,Airborne rhinovirus detection and effect of ul...
1,Discovering human history from stomach bacteria,Recent analyses of human pathogens have reveal...,Genome Biol,Discovering human history from stomach bacteri...
2,A new recruit for the army of the men of death,"The army of the men of death, in John Bunyan's...",Genome Biol,A new recruit for the army of the men of death...
3,Association of HLA class I with severe acute r...,BACKGROUND: The human leukocyte antigen (HLA) ...,BMC Med Genet,Association of HLA class I with severe acute r...
4,A double epidemic model for the SARS propagation,BACKGROUND: An epidemic of a Severe Acute Resp...,BMC Infect Dis,A double epidemic model for the SARS propagati...


##2. Obtain document vectors as matrices

In [0]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model = RobertaModel.from_pretrained('roberta-base').to(device)
model.eval()
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [0]:
def get_embed_matrix():
  save_path = f'{root_dir}outputs_hong/roberta.npy'
  try:
    return np.load(save_path)
  except:
    print(f'Cannot directly load numpy roberta.npy')
    token_info = tokenizer.batch_encode_plus(df['title+abstract'].tolist(), max_length=300, pad_to_max_length=True, return_attention_masks=True) 

    input_idss = torch.tensor(token_info['input_ids'], dtype=torch.long)
    attention_masks = torch.tensor(token_info['attention_mask'], dtype=torch.long)
    print(f'input tensor size: {input_idss.size()}')

    dataloader = DataLoader(TensorDataset(input_idss, attention_masks), batch_size=32, shuffle=False)
    # run model
    feature_vec = []
    with torch.no_grad():
      for input_ids, attention_mask in tqdm(dataloader):
        feature_vec.append(model(input_ids.to(device), attention_mask=attention_mask.to(device))[0][:,0,:].cpu())
    feature_vec = torch.cat(feature_vec, dim=0)
    print(f'output feature vec size: {feature_vec.size()}')
    np.save(save_path, feature_vec.numpy())
    return feature_vec

embed_matrix = get_embed_matrix()
embed_matrix.shape

In [0]:
# train tf-idf vectorizers and fit documents
tfidf_vectorizer = TfidfVectorizer(max_df=0.9, stop_words='english', max_features=800)

def get_dfidf_matrices():
  return tfidf_vectorizer.fit_transform(df['title+abstract'])


tfidf_matrix = get_dfidf_matrices()
print(f'tf-idf matrix shape: {tfidf_matrix.shape}')

tf-idf matrix shape: (42203, 800)


##3. Compute scores given a query

In [0]:
def query(query, method):

  if method == 'tf-idf':    
    vec = tfidf_vectorizer.transform([query]).toarray()[0] * 100
    matrix = tfidf_matrix.toarray() * 100
  else:
    with torch.no_grad():
      input_ids = torch.tensor(tokenizer.encode(query, add_special_tokens=True)).unsqueeze(0).to(device)
      vec = model(input_ids)[1].squeeze().cpu().numpy() * 100
    matrix = embed_matrix * 100
  
  # print(f'{method} distance: {1 - cosine(vec, matrix[0])}')

  score = np.asarray([1 - cosine(vec, doc_vec) for doc_vec in matrix])
  # print(f'{method}: {score}')
  return score

def score_document(vaccine_queries, therap_queries):
  def compute_other_score(a, b):
    sqrt_func = np.vectorize(math.sqrt)
    return 0.5 * (sqrt_func((1-a) * (1-b)) - sqrt_func((1+a) * (1+b)))
    # return -(a + b)

  vaccine_scoress, therap_scoress = [], []
  from itertools import product
  for method, vaccine_query, therap_query in product(['tf-idf'], vaccine_queries, therap_queries):
    vaccine_scoress.append(query(vaccine_query, method))
    therap_scoress.append(query(therap_query, method))

  final_scores = []
  for vaccine_scores, therap_scores in zip(vaccine_scoress, therap_scoress):
    other_scores = compute_other_score(vaccine_scores, therap_scores)
    # compute softmax scores
    scores = np.stack([other_scores, vaccine_scores, therap_scores], axis=0)
    scores = softmax(scores, axis=0)
    final_scores.append(scores)


  # majority votes
  final_votes = [np.argmax(final_score, axis=0) for final_score in final_scores]

  final_votes = np.stack(final_votes)
  print(f'finalvotes shape: {final_votes.shape}')
  print(final_votes[:,:20])
  from scipy.stats import mode
  final_votes = mode(final_votes, axis=0)[0].squeeze()
  print(f'finalvotes shape: {final_votes.shape}')
  return final_votes

In [0]:
vaccine_queries = ('vaccine vaccination dose antitoxin serum immunization inoculation for COVID-19 or coronavirus related research work',
                  'Methods evaluating potential complication of Antibody-Dependent Enhancement (ADE) in vaccine recipients of COVID-19 or coronavirus',
                   'Methods evaluating potential complication of Antibody-Dependent Enhancement (ADE) in vaccine recipients',
                   'Exploration of use of best animal models and their predictive value for a human vaccine')
therap_queries = ('Therapeutics treatment therapy drug antidotes cures remedies medication prophylactic restorative panacea for COVID-19 or coronavirus',
                   'Effectiveness of drugs like naproxen, clarithromycin, and minocyclinethat being developed that may exert effects on viral replication and tried to treat COVID-19 patients',
                  'Clinical and bench trials to investigate less common viral inhibitors against COVID-19 such as naproxen, clarithromycin, and minocyclinethat that may exert effects on viral replication.',
                  'Effectiveness of drugs being developed and tried to treat COVID-19 patients')
final_votes = score_document(vaccine_queries, therap_queries)


/usr/local/lib/python3.6/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
/usr/local/lib/python3.6/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
/usr/local/lib/python3.6/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
/usr/local/lib/python3.6/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
/usr/local/lib/python3.6/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)
/usr/local/lib/python3.6/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv

finalvotes shape: (16, 42203)
[[1 0 0 2 2 2 2 2 1 0 0 0 1 0 2 2 2 2 1 0]
 [2 0 0 1 2 2 1 2 1 2 0 0 1 2 1 2 1 1 1 2]
 [2 0 0 1 1 2 1 2 2 2 0 0 1 0 1 1 1 2 2 2]
 [2 0 0 1 2 1 1 1 1 2 0 0 1 0 1 2 1 1 1 2]
 [1 0 0 1 2 2 1 1 0 1 0 0 1 0 2 2 2 1 2 0]
 [2 0 0 1 2 2 1 2 0 2 0 0 1 2 1 2 1 1 1 2]
 [2 0 0 1 1 2 1 2 2 2 0 0 1 0 1 1 1 1 2 2]
 [2 0 0 1 2 1 1 1 0 2 0 0 1 0 2 2 1 1 1 2]
 [1 0 0 2 2 2 1 2 0 1 0 0 1 0 2 2 2 2 2 0]
 [2 0 0 2 2 2 1 2 0 2 0 0 1 2 1 2 0 1 2 2]
 [2 0 0 1 0 2 1 2 2 2 0 0 1 0 1 1 0 2 2 2]
 [2 0 0 2 2 0 1 1 0 2 0 0 1 0 1 2 0 1 0 2]
 [0 1 0 2 2 2 2 2 1 0 0 0 1 0 2 2 2 2 2 0]
 [2 1 0 1 2 2 0 2 1 2 0 0 1 2 2 2 0 2 2 2]
 [2 1 0 1 0 2 0 2 2 2 0 0 1 0 2 2 0 2 2 2]
 [2 1 0 1 2 0 0 0 1 2 0 0 1 0 2 2 0 1 0 2]]
finalvotes shape: (42203,)


In [0]:
from collections import Counter
print(Counter(final_votes))
final_votes[:20]


Counter({2: 15477, 0: 13692, 1: 13034})


array([2, 0, 0, 1, 2, 2, 1, 2, 0, 2, 0, 0, 1, 0, 1, 2, 0, 1, 2, 2])

In [0]:
# save as json
import json
json_dict = {}
for i, row in df.iterrows():
  json_dict[row['title']] = int(final_votes[i])
with open('/content/drive/Shared drives/COVID-19-Research/results/Hong/votes.json', 'w') as fp:
    json.dump(json_dict, fp)


In [0]:
save_path = '/content/drive/Shared drives/COVID-19-Research/results/Hong/votes'
np.save(save_path, final_votes)

In [0]:
def get_score(title):
  index = df.index[df['title'] == title].tolist()
  
  print(final_scores[index[0]])

In [0]:
df.iloc[24]['title']

In [0]:
get_score('Immune-mediated diseases of the dog and cat III. Immune-mediated diseases of the integumentary, urogenital, endocrine and vascular systems')